# Lab 3: Data Operations with Strings

**Duration:** 45 minutes  
**Objective:** Master string operations for policy numbers, customer IDs, and data management

## 🎯 Learning Objectives

- Implement policy number generation and validation
- Perform atomic premium calculations
- Manage customer data with string operations
- Build document processing workflows
- Optimize batch operations for policy updates

## Part 1: Setup and Basic String Operations (10 minutes)

In [ ]:
import redis
import json
import time
import random
import string
from datetime import datetime, timedelta
from colorama import init, Fore, Style
import pandas as pd

init(autoreset=True)

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Clear existing data
r.flushdb()
print(f"{Fore.GREEN}✅ Connected to Redis and cleared database{Style.RESET_ALL}")

In [ ]:
class PolicyManager:
    """Manage policy operations using Redis strings."""
    
    def __init__(self, redis_client):
        self.r = redis_client
        self.policy_prefix = 'policy'
        self.counter_key = 'policy:counter:2024'
    
    def generate_policy_number(self, policy_type='AUTO'):
        """Generate unique policy number using atomic counter."""
        # Atomic increment for unique ID
        policy_id = self.r.incr(self.counter_key)
        
        # Format: POL-TYPE-YEAR-NUMBER
        year = datetime.now().year
        policy_number = f"POL-{policy_type}-{year}-{policy_id:06d}"
        
        return policy_number
    
    def create_policy(self, customer_id, policy_type, premium, coverage):
        """Create a new policy with all details."""
        policy_number = self.generate_policy_number(policy_type)
        
        # Store policy details
        policy_data = {
            f'{self.policy_prefix}:{policy_number}:customer': customer_id,
            f'{self.policy_prefix}:{policy_number}:type': policy_type,
            f'{self.policy_prefix}:{policy_number}:premium': str(premium),
            f'{self.policy_prefix}:{policy_number}:coverage': str(coverage),
            f'{self.policy_prefix}:{policy_number}:status': 'ACTIVE',
            f'{self.policy_prefix}:{policy_number}:created': datetime.now().isoformat()
        }
        
        # Use MSET for atomic operation
        self.r.mset(policy_data)
        
        return policy_number
    
    def get_policy(self, policy_number):
        """Retrieve policy details."""
        keys = [
            f'{self.policy_prefix}:{policy_number}:customer',
            f'{self.policy_prefix}:{policy_number}:type',
            f'{self.policy_prefix}:{policy_number}:premium',
            f'{self.policy_prefix}:{policy_number}:coverage',
            f'{self.policy_prefix}:{policy_number}:status',
            f'{self.policy_prefix}:{policy_number}:created'
        ]
        
        values = self.r.mget(keys)
        
        if not any(values):
            return None
        
        return {
            'policy_number': policy_number,
            'customer': values[0],
            'type': values[1],
            'premium': float(values[2]) if values[2] else 0,
            'coverage': float(values[3]) if values[3] else 0,
            'status': values[4],
            'created': values[5]
        }

# Test policy management
pm = PolicyManager(r)

print(f"{Fore.CYAN}=== POLICY MANAGEMENT ==={Style.RESET_ALL}\n")

# Create multiple policies
policies = []
for i in range(5):
    policy_num = pm.create_policy(
        customer_id=f"CUST-{1000+i}",
        policy_type=random.choice(['AUTO', 'HOME', 'LIFE']),
        premium=random.randint(500, 2000),
        coverage=random.randint(10000, 100000)
    )
    policies.append(policy_num)
    print(f"Created: {policy_num}")

# Retrieve and display a policy
print(f"\n{Fore.CYAN}Policy Details:{Style.RESET_ALL}")
policy_details = pm.get_policy(policies[0])
for key, value in policy_details.items():
    print(f"  {key}: {value}")

## Part 2: Atomic Operations and Counters (15 minutes)

In [ ]:
class PremiumCalculator:
    """Handle premium calculations with atomic operations."""
    
    def __init__(self, redis_client):
        self.r = redis_client
    
    def adjust_premium(self, policy_number, adjustment):
        """Atomically adjust premium amount."""
        key = f'policy:{policy_number}:premium'
        
        # Use INCRBYFLOAT for atomic float operations
        new_premium = self.r.incrbyfloat(key, adjustment)
        return float(new_premium)
    
    def apply_discount(self, policy_number, discount_percent):
        """Apply percentage discount to premium."""
        key = f'policy:{policy_number}:premium'
        
        # Get current premium
        current = float(self.r.get(key) or 0)
        
        # Calculate discount
        discount_amount = current * (discount_percent / 100)
        
        # Apply discount atomically
        new_premium = self.r.incrbyfloat(key, -discount_amount)
        
        return {
            'original': current,
            'discount': discount_amount,
            'new_premium': float(new_premium)
        }
    
    def track_claims(self, policy_number):
        """Track number of claims for a policy."""
        claims_key = f'policy:{policy_number}:claims:count'
        total_key = f'policy:{policy_number}:claims:total'
        
        # Increment claim count
        claim_count = self.r.incr(claims_key)
        
        # Add to total claims amount
        claim_amount = random.randint(1000, 10000)
        total_claims = self.r.incrbyfloat(total_key, claim_amount)
        
        return {
            'claim_number': claim_count,
            'claim_amount': claim_amount,
            'total_claims': float(total_claims)
        }

# Test atomic operations
calc = PremiumCalculator(r)

print(f"{Fore.CYAN}=== ATOMIC PREMIUM OPERATIONS ==={Style.RESET_ALL}\n")

# Test premium adjustments
test_policy = policies[0]
print(f"Testing with policy: {test_policy}\n")

# Apply various adjustments
print("Premium Adjustments:")
new_premium = calc.adjust_premium(test_policy, 50)
print(f"  Added late fee: ${new_premium:.2f}")

new_premium = calc.adjust_premium(test_policy, -25)
print(f"  Applied credit: ${new_premium:.2f}")

# Apply discount
discount_result = calc.apply_discount(test_policy, 10)
print(f"\nDiscount Applied (10%):")
print(f"  Original: ${discount_result['original']:.2f}")
print(f"  Discount: ${discount_result['discount']:.2f}")
print(f"  New Premium: ${discount_result['new_premium']:.2f}")

# Track claims
print(f"\n{Fore.CYAN}Claims Tracking:{Style.RESET_ALL}")
for i in range(3):
    claim = calc.track_claims(test_policy)
    print(f"  Claim #{claim['claim_number']}: ${claim['claim_amount']:,} (Total: ${claim['total_claims']:,.2f})")

## Part 3: Batch Operations and Performance (10 minutes)

In [ ]:
class BatchProcessor:
    """Handle batch operations for multiple policies."""
    
    def __init__(self, redis_client):
        self.r = redis_client
    
    def bulk_create_customers(self, count=100):
        """Create multiple customers in batch."""
        print(f"Creating {count} customers...")
        
        # Prepare batch data
        customer_data = {}
        customer_ids = []
        
        for i in range(count):
            cust_id = f"CUST-{10000+i}"
            customer_ids.append(cust_id)
            
            customer_data[f'customer:{cust_id}:name'] = f"Customer {i}"
            customer_data[f'customer:{cust_id}:email'] = f"customer{i}@example.com"
            customer_data[f'customer:{cust_id}:score'] = str(random.randint(600, 850))
        
        # Batch insert
        start_time = time.perf_counter()
        self.r.mset(customer_data)
        end_time = time.perf_counter()
        
        elapsed = (end_time - start_time) * 1000
        print(f"✅ Created {count} customers in {elapsed:.2f}ms")
        print(f"   Rate: {count/(elapsed/1000):.0f} customers/second")
        
        return customer_ids
    
    def bulk_update_status(self, pattern, new_status):
        """Update status for multiple policies."""
        # Find matching keys
        keys = self.r.keys(f'{pattern}:status')
        
        if not keys:
            return 0
        
        # Prepare update data
        update_data = {key: new_status for key in keys}
        
        # Batch update
        self.r.mset(update_data)
        
        return len(keys)
    
    def performance_test(self):
        """Test performance of different operations."""
        results = []
        
        # Test 1: Individual SET operations
        start = time.perf_counter()
        for i in range(1000):
            self.r.set(f'perf:individual:{i}', f'value{i}')
        individual_time = (time.perf_counter() - start) * 1000
        
        # Test 2: Batch MSET operation
        batch_data = {f'perf:batch:{i}': f'value{i}' for i in range(1000)}
        start = time.perf_counter()
        self.r.mset(batch_data)
        batch_time = (time.perf_counter() - start) * 1000
        
        # Test 3: Pipeline operations
        start = time.perf_counter()
        pipe = self.r.pipeline()
        for i in range(1000):
            pipe.set(f'perf:pipeline:{i}', f'value{i}')
        pipe.execute()
        pipeline_time = (time.perf_counter() - start) * 1000
        
        # Clean up
        for prefix in ['perf:individual:*', 'perf:batch:*', 'perf:pipeline:*']:
            for key in self.r.keys(prefix):
                self.r.delete(key)
        
        return {
            'individual': individual_time,
            'batch': batch_time,
            'pipeline': pipeline_time
        }

# Test batch operations
batch = BatchProcessor(r)

print(f"{Fore.CYAN}=== BATCH OPERATIONS ==={Style.RESET_ALL}\n")

# Create customers in batch
customer_ids = batch.bulk_create_customers(100)

# Update multiple statuses
updated = batch.bulk_update_status('policy:*', 'RENEWED')
print(f"\n✅ Updated {updated} policy statuses")

# Performance comparison
print(f"\n{Fore.CYAN}Performance Comparison (1000 operations):{Style.RESET_ALL}")
perf_results = batch.performance_test()

print(f"  Individual SETs: {perf_results['individual']:.2f}ms")
print(f"  Batch MSET:      {perf_results['batch']:.2f}ms")
print(f"  Pipeline:        {perf_results['pipeline']:.2f}ms")
print(f"\n  Speedup (MSET):    {perf_results['individual']/perf_results['batch']:.1f}x faster")
print(f"  Speedup (Pipeline): {perf_results['individual']/perf_results['pipeline']:.1f}x faster")

## Part 4: Advanced String Patterns (10 minutes)

In [ ]:
# Advanced string patterns
print(f"{Fore.CYAN}=== ADVANCED STRING PATTERNS ==={Style.RESET_ALL}\n")

# Pattern 1: Distributed Locking
def acquire_lock(key, timeout=10):
    """Acquire a distributed lock."""
    lock_key = f"lock:{key}"
    identifier = str(time.time())
    
    # Try to acquire lock with NX (only if not exists) and EX (expiration)
    acquired = r.set(lock_key, identifier, nx=True, ex=timeout)
    
    if acquired:
        return identifier
    return None

def release_lock(key, identifier):
    """Release a distributed lock."""
    lock_key = f"lock:{key}"
    
    # Only release if we own the lock
    if r.get(lock_key) == identifier:
        r.delete(lock_key)
        return True
    return False

# Test locking
print("Testing Distributed Locking:")
lock_id = acquire_lock("resource:1")
if lock_id:
    print(f"  ✅ Lock acquired: {lock_id}")
    
    # Try to acquire same lock (should fail)
    lock_id2 = acquire_lock("resource:1")
    if not lock_id2:
        print("  ❌ Second lock attempt failed (expected)")
    
    # Release lock
    if release_lock("resource:1", lock_id):
        print("  ✅ Lock released")

# Pattern 2: Rate Limiting
def check_rate_limit(user_id, max_requests=10, window=60):
    """Check if user has exceeded rate limit."""
    key = f"rate:{user_id}:{int(time.time()/window)}"
    
    # Increment counter
    current = r.incr(key)
    
    # Set expiration on first request
    if current == 1:
        r.expire(key, window)
    
    return current <= max_requests

print("\nTesting Rate Limiting:")
user = "user:123"
for i in range(12):
    allowed = check_rate_limit(user, max_requests=10)
    if allowed:
        print(f"  Request {i+1}: ✅ Allowed")
    else:
        print(f"  Request {i+1}: ❌ Rate limited")

# Pattern 3: Conditional Updates
print("\nTesting Conditional Updates:")

# Set initial value
r.set("balance:account:1", "1000")
print(f"  Initial balance: $1000")

# Attempt withdrawal with check
def withdraw(account, amount):
    key = f"balance:{account}"
    current = float(r.get(key) or 0)
    
    if current >= amount:
        new_balance = r.incrbyfloat(key, -amount)
        return True, new_balance
    return False, current

# Test withdrawals
success, balance = withdraw("account:1", 300)
print(f"  Withdraw $300: {'✅' if success else '❌'} Balance: ${balance:.2f}")

success, balance = withdraw("account:1", 800)
print(f"  Withdraw $800: {'✅' if success else '❌'} Balance: ${balance:.2f}")